In [317]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from keras.models import Sequential
from keras.layers import Dense

import re

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

import gensim
from gensim import corpora

In [318]:
"""
define a function for auto ml
define a function to prepare data
define a function to clean text (also define stopwords, lemmatizer, punctuations)
"""

def auto_ML(model, Xtrain, Ytrain, Xtest, Ytest, folds= 10):
    model.fit(Xtrain, Ytrain)
    prediction= model.predict(Xtest)
    
    # Cross validation
    scores= cross_val_score(model, Xtrain, Ytrain, cv= folds)
    val_score= sum(scores/folds)
    
    print('Test prediction:\n{}'.format(prediction))
    print('-------------------------------------')
    print('Accuracy score: {}'.format(accuracy_score(Ytest, prediction)))
    print('-------------------------------------')
    print('confusion matrix:\n {}'.format(confusion_matrix(Ytest, prediction)))
    print('-------------------------------------')
    print('Cross Validation score: {}'.format(val_score))
    
    
def prepare_data(data, split_ratio):
    split= int(data.shape[0]*split_ratio)
    
    X_train= data.loc[:split, :]
    X_test= data.loc[split: , :]
    
    print('train shape: {}'.format(X_train.shape))
    print('test shape: {}'.format(X_test.shape))
    
    return X_train, X_test



stop= stopwords.words('english')
exclude = string.punctuation 
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free= ' '.join(word for word in doc.lower().split() if word not in stop)
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized
    

In [346]:
train= pd.read_csv('/Users/akshatpant/Desktop/UMD/Sem 3/Comp Ling/Project/train.csv')
test= pd.read_csv('/Users/akshatpant/Desktop/UMD/Sem 3/Comp Ling/Project/test.csv')

In [336]:
train.head()

,Question ID,Question Text,QANTA Scores,Answer,Sentence Position,IR_Wiki Scores,category
0,2390,it was founded as kart-hadasht meaning new tow...,"portugal:0.0748547872365, peter_the_great:0.05...",carthage,0,"carthage:4.46484593046, nelson_mandela:3.46234...",history
1,2390,it was founded as kart-hadasht meaning new tow...,"battle_of_shiloh:0.033232900633, henry_the_nav...",carthage,1,"carthage:6.0041482036, ancient_corinth:4.56760...",history
2,2390,it was founded as kart-hadasht meaning new tow...,"hannibal:0.0323164945129, carthage:0.027918372...",carthage,2,"carthage:6.40252637044, ancient_corinth:5.3011...",history
3,2390,it was founded as kart-hadasht meaning new tow...,"carthage:0.120553743404, hannibal:0.0341056783...",carthage,3,"carthage:6.74579468525, ancient_corinth:4.3816...",history
4,106503,the nations involved in it had allied immediat...,"battle_of_austerlitz:0.06581875769, treaty_of_...",congress_of_vienna,0,"congress_of_vienna:3.78992924717, crimean_war:...",history


In [337]:
print('training data:')
train.shape

training data:


(8160, 7)

In [338]:
print('testing data: ')
test.shape

testing data: 


(306, 6)

### add new features

In [347]:
"""
Add new feature: Question length

Add new features: Extract max IR score and correscponding page from IR_Wiki Scores
Also calculate difference between highest and second highest IR score

Add target feature paren_match (true if wiki page and answer match)

We will add the following features after splitting into training and testing.
1. Apply Weight of Evidence(WOE) encoding to category variable
2. Topic modeling
"""

#Question length (training data)
lens= []
for i in range(0, train.shape[0]):
    lens.append(len(train.loc[i]['Question Text']))
lens
train['Quest len']= lens
print('Question length created for training data')

#Question length (testing data)
lens= []
for i in range(0, test.shape[0]):
    lens.append(len(test.loc[i]['Question Text']))
lens
test['Quest len']= lens
print('Question length created for testing data')



######################################################################

# Max IR score and corresponding page (training data)
wiki_page= []
page_score= []
diff= []
for i in range(0, train.shape[0]):
    ans_score= {}
    for ii in train.loc[i]['IR_Wiki Scores'].split(', '):
        ans_score[ii.split(':')[0]]= float(ii.split(':')[1])
        
    
    page= sorted(ans_score, key= ans_score.get, reverse= True)[0]
    page2= sorted(ans_score, key= ans_score.get, reverse= True)[1]
    wiki_page.append(page)
    page_score.append(ans_score[page])
    diff.append(ans_score[page]- ans_score[page2])

train['Wiki_page']= wiki_page
train['Wiki_page_embed']= wiki_page
train['Page score']= page_score
train['Score difference']= diff
print('Max IR score and corresponding page created for training data')



# Max IR score and corresponding page (testing data)
wiki_page= []
page_score= []
diff= []
for i in range(0, test.shape[0]):
    ans_score= {}
    for ii in test.loc[i]['IR_Wiki Scores'].split(', '):
        ans_score[ii.split(':')[0]]= float(ii.split(':')[1])
        
    
    page= sorted(ans_score, key= ans_score.get, reverse= True)[0]
    page2= sorted(ans_score, key= ans_score.get, reverse= True)[1]
    wiki_page.append(page)
    page_score.append(ans_score[page])
    diff.append(ans_score[page]- ans_score[page2])

test['Wiki_page']= wiki_page
test['Wiki_page_embed']= wiki_page
test['Page score']= page_score
test['Score difference']= diff
print('Max IR score and corresponding page created for testing data')



################################################################

# Target feature paren_match (training data). It is 1 if answer and wiki page match. 0 otherwise
train['paren_match']= 0

for i, row in train.iterrows():
    if row['Answer'] == row['Wiki_page']:
        train.loc[i, 'paren_match']= 1
print('paren_match created for training data')



#################################################################

# Create a new column to embed categories (training data)
train['category_embed']= train['category']
encoding= ce.WOEEncoder(cols= ['category_embed'], impute_missing= True)
encoding.fit(train[['category_embed']], train['paren_match'])
train['category_embed']=encoding.transform(train[['category_embed']])
print('Category embeddings created for training data')



# Create a new column to embed categories (testing data)
test['category_embed']= test['category']
test['category_embed']= encoding.transform(test[['category_embed']])
print('Category embeddings created for testing data')




#################################################################

# Create new topic columns for topic modeling (training data)
train['Topic 0']= 0
train['Topic 1']= 0
train['Topic 2']= 0
train['Topic 3']= 0
print('Topics created for training data')


# Create new topic columns for topic modeling (testing data)
test['Topic 0']= 0
test['Topic 1']= 0
test['Topic 2']= 0
test['Topic 3']= 0
print('Topics created for testing data')


##################################################################
##################################################################

"""
Add new feature: topics (topic modeling)
We have 4 categories in the dataset. SO lets use 4 topics
"""
# Preprocessing the text first
clean_questions= [clean(row['Question Text']).split() for _, row in train.iterrows()]
print('Text preprocessing done')

# Create a word to index mapping
dictionary= corpora.Dictionary(clean_questions)
print('Word index dictionary created')

# Convert the list of documents to a BOW matrix using the dictionary prepared above
doc_term_matrix = [dictionary.doc2bow(doc) for doc in clean_questions]
print('BOW matrix created')

# Train the LDA model on the document word matrix
ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50)
print('LDA model trained')


"""
Add topic distributions as features to train and test data
"""
# Topic distributions (training data)
for i, row in train.iterrows():
    for pair in ldamodel[doc_term_matrix[i]]:
        col= 'Topic '+ str(pair[0])
        train.loc[i, col]= pair[1]
        
print('Topic distributions added to training data')    
    

# Topic distributions (testing data)

clean_test= [clean(row['Question Text']).split() for _, row in test.iterrows()]
test_bow= [dictionary.doc2bow(doc) for doc in clean_test]

for i, row in test.iterrows():
    for pair in ldamodel.get_document_topics(test_bow[i]):
        col= 'Topic '+ str(pair[0])
        test.loc[i, col]= pair[1]

print('Topic distributions added to testing data')

Question length created for training data
Question length created for testing data
Max IR score and corresponding page created for training data
Max IR score and corresponding page created for testing data
paren_match created for training data
Category embeddings created for training data
Category embeddings created for testing data
Topics created for training data
Topics created for testing data
Text preprocessing done
Word index dictionary created
BOW matrix created
LDA model trained
Topic distributions added to training data
Topic distributions added to testing data


## Split into training and testing

In [296]:
X_train, X_test, y_train, y_test = train_test_split(train, train['paren_match'], test_size= 0.2, random_state= 4)
print('X train shape: {}'.format(X_train.shape))
print('y train shape: {}'.format(y_train.shape))
print('X test shape: {}'.format(X_test.shape))
print('y test shape: {}'.format(y_test.shape))

X_train= X_train.reset_index(drop= True)
y_train= y_train.reset_index(drop= True)
X_test= X_test.reset_index(drop= True)
y_test= y_test.reset_index(drop= True)

print('reindexed X train and y train for WOE embeddings')
print('reindexed X test and y test for WOE embeddings')

X train shape: (6528, 18)
y train shape: (6528,)
X test shape: (1632, 18)
y test shape: (1632,)
reindexed X train and y train for WOE embeddings
reindexed X test and y test for WOE embeddings


In [298]:
"""
Add new feature: topics (topic modeling)
We have 4 categories in the dataset. SO lets use 4 topics
"""
# Preprocessing the text first
clean_questions= [clean(row['Question Text']).split() for _, row in X_train.iterrows()]
print('Text preprocessing done')

# Create a word to index mapping
dictionary= corpora.Dictionary(clean_questions)
print('Word index dictionary created')

# Convert the list of documents to a BOW matrix using the dictionary prepared above
doc_term_matrix = [dictionary.doc2bow(doc) for doc in clean_questions]
print('BOW matrix created')

# Train the LDA model on the document word matrix
ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50)
print('LDA model trained')

Text preprocessing done
Word index dictionary created
BOW matrix created
LDA model trained


In [300]:
"""
Add topic distributions as features to train and test data
"""
# Topic distributions (training data)
for i, row in X_train.iterrows():
    for pair in ldamodel[doc_term_matrix[i]]:
        col= 'Topic '+ str(pair[0])
        X_train.loc[i, col]= pair[1]
        
print('Topic distributions added to training data')    
    

# Topic distributions (testing data)

clean_test= [clean(row['Question Text']).split() for _, row in X_test.iterrows()]
new_bow= [dictionary.doc2bow(doc) for doc in clean_test]

for i, row in X_test.iterrows():
    for pair in ldamodel.get_document_topics(new_bow[i]):
        col= 'Topic '+ str(pair[0])
        X_test.loc[i, col]= pair[1]

print('Topic distributions added to testing data')

Topic distributions added to training data
Topic distributions added to testing data


### features

In [348]:
features= ['Quest len', 'Page score',  'category_embed', 'Score difference', 'Topic 0', 'Topic 1', 'Topic 2', 'Topic 3']
target= ['paren_match']

### regularize data

In [349]:
feat_reg= ['Quest len', 'Page score',  'category_embed', 'Score difference']

scaler= StandardScaler()

scaler.fit(train[feat_reg].values)

train[feat_reg]= scaler.transform(train[feat_reg].values)
test[feat_reg]= scaler.transform(test[feat_reg].values)

print('regularized X train and X test')

y_train= np.reshape(train['paren_match'].values, (train['paren_match'].shape[0], ))

regularized X train and X test


### SVM

In [358]:
svm_clf= SVC(kernel= 'rbf')
svm_clf.fit(train[features], train['paren_match'])
prediction= model.predict(test[features])

In [359]:
prediction

array([[0.37693372],
       [0.9273552 ],
       [0.5692925 ],
       [0.92153156],
       [0.9887227 ],
       [0.98157823],
       [0.7829033 ],
       [0.39093325],
       [0.99340856],
       [0.99442285],
       [0.24382211],
       [0.9986193 ],
       [0.91632587],
       [0.27735618],
       [0.99976295],
       [0.7344661 ],
       [0.999642  ],
       [0.99893814],
       [0.998936  ],
       [0.9376285 ],
       [0.9935383 ],
       [0.7951771 ],
       [0.97926396],
       [0.2221207 ],
       [0.99554765],
       [0.77631724],
       [0.71622723],
       [0.65036094],
       [0.26146162],
       [0.6444323 ],
       [0.34097007],
       [0.43532774],
       [0.5230464 ],
       [0.31744567],
       [0.9999298 ],
       [0.2432496 ],
       [0.8977545 ],
       [0.99582434],
       [0.9998136 ],
       [0.9398417 ],
       [0.27157182],
       [0.99426574],
       [0.6373742 ],
       [0.98854476],
       [0.9962088 ],
       [0.96416414],
       [0.40895152],
       [0.994

In [362]:
test['buzz']= 'false'

for i, row in test.iterrows():
    if prediction[i]>= 0.5:
        test.loc[i, 'buzz']= 'true'

In [363]:
test.head()

,Question ID,Question Text,QANTA Scores,Sentence Position,IR_Wiki Scores,category,Quest len,Wiki_page,Wiki_page_embed,Page score,Score difference,category_embed,Topic 0,Topic 1,Topic 2,Topic 3,buzz
0,165921,this group 's power peaked in the 31st congres...,"ronald_reagan:0.0484481338785, grover_clevelan...",1,"free_soil_party:2.87910481765, equal_rights_am...",history,-0.743353,free_soil_party,free_soil_party,-0.648116,-0.524271,1.045459,0.091492,0.186114,0.012338,0.710055,false
1,165935,charles lorencez led 6000 troops in a frontal ...,"battle_of_puebla:0.835245616606, francisco_i._...",1,"battle_of_puebla:5.78287582417, pancho_villa:3...",history,-0.837524,battle_of_puebla,battle_of_puebla,-0.589664,-0.393370,1.045459,0.012416,0.113529,0.012883,0.861172,true
2,102459,in 1866 this mans nation under the command of ...,"vitus_bering:0.0350534508237, verdun:0.0287764...",0,"charles_de_gaulle:4.06032122098, napoleon_iii:...",history,-0.842755,charles_de_gaulle,charles_de_gaulle,-0.624338,-0.478861,1.045459,0.013854,0.203853,0.160359,0.621934,true
3,4179,they became prominent after intervening in and...,"teutonic_knights:0.0501695382695, otto_von_bis...",4,"teutonic_knights:7.01499316116, golden_horde:4...",history,2.406122,teutonic_knights,teutonic_knights,-0.564862,-0.368264,1.045459,0.119033,0.047699,0.000000,0.828114,true
4,4186,its capital may have been established as early...,"songhai_empire:0.0943808941563, ming_dynasty:0...",2,"songhai_empire:6.52974316335, mali_empire:3.50...",history,0.789531,songhai_empire,songhai_empire,-0.574630,-0.352887,1.045459,0.054202,0.159999,0.000000,0.779786,true


In [364]:
test.to_csv('/Users/akshatpant/Desktop/UMD/Sem 3/Comp Ling/Project/predict.csv')

In [355]:
svm_clf= SVC(kernel= 'rbf')

auto_ML(svm_clf, train[features], train['paren_match'], test[features], y_test, folds= 10)

Test prediction:
[0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 1 1 0 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 0 0 1 0 1 1 0 1 1 0 0 1 1 0 1 1
 1 1 1 1 1 1 0 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0
 1 1 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1
 1 0 1 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 0 0 1 0 1 1
 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 0 0 1 0 0]
-------------------------------------


ValueError: Found input variables with inconsistent numbers of samples: [1632, 306]

### Feedforward net

In [308]:
"""
Prepare input
"""
X_train= np.array(X_train[features])
y_train= np.array(y_train)

X_test= np.array(X_test[features])
y_test= np.array(y_test)


In [309]:
"""
Create model
"""
model= Sequential()
model.add(Dense(15, activation='relu'))
model.add(Dense(10, activation= 'relu'))
model.add(Dense(5, activation= 'relu'))
model.add(Dense(1, activation= 'sigmoid'))

"""
compile model
"""
model.compile(loss= 'binary_crossentropy', optimizer= 'adam')

"""
Fit the model
"""
model.fit(X_train, y_train, epochs=20, batch_size=10, verbose=1, validation_split= 0.2)

"""
Calculate predictions
"""
predictions= model.predict(X_test)

"""
calculate accuracy
print confusion matrix

         Pred labels
         0     1
Target 0
labels 1

"""
#threshold to decide class
threshold= 0.5

count= 0
false_positive= 0
false_negative= 0
true_positive= 0
true_negative= 0
for i in range(0, predictions.shape[0]):
    if (predictions[i]> threshold) & (y_test[i]== 1):
        count+= 1
        true_positive+= 1
    if (predictions[i]<= threshold) & (y_test[i]== 0):
        count+= 1
        true_negative+= 1
    if (predictions[i]> threshold) & (y_test[i]== 0):
        false_positive+= 1
    if (predictions[i]<= threshold) & (y_test[i]== 1):
        false_negative+= 1
        
print('\tPred Labels\n\t0\t1\nTarget 0 {}\t{}\nlabels 1 {}\t{}'.format(true_negative, 
                                false_positive, false_negative, true_positive))

print('\n')

print(count/predictions.shape[0])



Train on 5222 samples, validate on 1306 samples
Epoch 1/20
5222/5222 [==============================] - 1s 280us/step - loss: 0.5401 - val_loss: 0.5004
Epoch 2/20
5222/5222 [==============================] - 1s 133us/step - loss: 0.4649 - val_loss: 0.4417
Epoch 3/20
5222/5222 [==============================] - 1s 134us/step - loss: 0.4252 - val_loss: 0.4073
Epoch 4/20
5222/5222 [==============================] - 1s 131us/step - loss: 0.4013 - val_loss: 0.3850
Epoch 5/20
5222/5222 [==============================] - 1s 127us/step - loss: 0.3881 - val_loss: 0.3731
Epoch 6/20
5222/5222 [==============================] - 1s 131us/step - loss: 0.3782 - val_loss: 0.3629
Epoch 7/20
5222/5222 [==============================] - 1s 177us/step - loss: 0.3714 - val_loss: 0.3597
Epoch 8/20
5222/5222 [==============================] - 1s 142us/step - loss: 0.3702 - val_loss: 0.3550
Epoch 9/20
5222/5222 [==============================] - 1s 129us/step - loss: 0.3640 - val_loss: 0.3591
Epoch 10/20
5222

In [42]:
X_train, X_test, y_train, y_test = train_test_split(train_df, train_df[target], test_size=0.20)

In [13]:
def prepare_seq(seq, to_ix):
    idxs= []
    for w in seq:
        if w in to_ix:
            idxs.append(to_ix[w])
        else:
            idxs.append(to_ix['<UNK>'])
    return torch.tensor(idxs, dtype=torch.long)

In [43]:
"""
prepare word_to_index dictionary
set <pad> and <unk> value to 0
normalize text before adding to vocab
remove stopwords
"""

regex= re.compile(r"\b(\w*['\w]*[\w]*)[^\w]*")


word_to_ix= {'<UNK>': 0}

for i, row in train_df.iterrows():
    sent= train_df.loc[i, 'Question Text'].lower()
    for word in regex.findall(sent):
        if word not in word_to_ix:
            word_to_ix[word]= len(word_to_ix)- 1
    train_df.loc[i, 'Question Text']= ' '.join(regex.findall(sent))
    
word_to_ix


/Users/akshatpant/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


{'<UNK>': 0,
 'the': 0,
 'name': 1,
 'of': 2,
 'village': 3,
 'for': 4,
 'which': 5,
 'this': 6,
 'battle': 7,
 'is': 8,
 'named': 9,
 'derived': 10,
 'from': 11,
 'a': 12,
 'small': 13,
 'red': 14,
 'flowering': 15,
 'tree': 16,
 'problems': 17,
 'with': 18,
 'french': 19,
 'interference': 20,
 'and': 21,
 'illegal': 22,
 'use': 23,
 'trade': 24,
 'permits': 25,
 'precipitated': 26,
 'began': 27,
 'shortly': 28,
 'after': 29,
 'capture': 30,
 'fort': 31,
 'william': 32,
 '': 33,
 'winning': 34,
 'side': 35,
 'lost': 36,
 'only': 37,
 '50': 38,
 'men': 39,
 'compared': 40,
 'to': 41,
 '1200': 42,
 'casualties': 43,
 'losers': 44,
 'whose': 45,
 'commander': 46,
 'escaped': 47,
 'but': 48,
 'had': 49,
 '30000': 50,
 'pound': 51,
 'bounty': 52,
 'placed': 53,
 'on': 54,
 'his': 55,
 'head': 56,
 'based': 57,
 'intelligence': 58,
 'lord': 59,
 'george': 60,
 'murray': 61,
 'eventual': 62,
 'winners': 63,
 'took': 64,
 'up': 65,
 'position': 66,
 'where': 67,
 'opposition': 68,
 'was': 69,

In [15]:
train_df

,Question ID,Question Text,QANTA Scores,Answer,Sentence Position,IR_Wiki Scores,category,Quest len,Wiki page,Page score,Score difference,paren_match
0,2390,it was founded as kart hadasht meaning new tow...,"portugal:0.0748547872365, peter_the_great:0.05...",carthage,0,"carthage:4.46484593046, nelson_mandela:3.46234...",1.045459,-1.397314,0.854227,-0.616195,-0.474830,1
1,2390,it was founded as kart hadasht meaning new tow...,"battle_of_shiloh:0.033232900633, henry_the_nav...",carthage,1,"carthage:6.0041482036, ancient_corinth:4.56760...",1.045459,-1.057254,0.854227,-0.585210,-0.448684,1
2,2390,it was founded as kart hadasht meaning new tow...,"hannibal:0.0323164945129, carthage:0.027918372...",carthage,2,"carthage:6.40252637044, ancient_corinth:5.3011...",1.045459,-0.078929,0.854227,-0.577191,-0.468873,1
3,2390,it was founded as kart hadasht meaning new tow...,"carthage:0.120553743404, hannibal:0.0341056783...",carthage,3,"carthage:6.74579468525, ancient_corinth:4.3816...",1.045459,0.533178,0.854227,-0.570281,-0.392808,1
4,106503,the nations involved in it had allied immediat...,"battle_of_austerlitz:0.06581875769, treaty_of_...",congress_of_vienna,0,"congress_of_vienna:3.78992924717, crimean_war:...",1.045459,-0.984011,0.854227,-0.629781,-0.479407,1
5,106503,the nations involved in it had allied immediat...,"treaty_of_brest-litovsk:0.0290714511706, congr...",congress_of_vienna,1,"congress_of_vienna:3.91160914684, napoleon_iii...",1.045459,-0.288197,0.854227,-0.627332,-0.477003,1
6,106503,the nations involved in it had allied immediat...,"congress_of_vienna:0.18004693222, teutonic_kni...",congress_of_vienna,2,"congress_of_vienna:7.0196528341, thirty_years'...",1.045459,0.418081,0.854227,-0.564768,-0.299050,1
7,106503,the nations involved in it had allied immediat...,"congress_of_vienna:0.224345119428, teutonic_kn...",congress_of_vienna,3,"congress_of_vienna:6.65354658301, napoleon_iii...",1.045459,0.705824,0.854227,-0.572138,-0.333895,1
8,165897,at age 22 he became the youngest captain in th...,"benito_mussolini:0.343499429074, hannibal:0.09...",francisco_franco,0,"seminole_wars:3.40736769424, francisco_franco:...",1.045459,-1.549033,0.112876,-0.637482,-0.534471,0
9,165897,at age 22 he became the youngest captain in th...,"emilio_aguinaldo:0.0391431785257, napoleon_iii...",francisco_franco,1,"francisco_franco:5.81715654207, spanish_civil_...",1.045459,-0.670110,-0.193592,-0.588974,-0.392120,1


In [36]:
"""
Create LSTM network for the text.
Combine other features into the output of LSTM.

"""

class LSTM_special(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, batch_size, vocab_size, add_dim, out_dim= 1, num_layers= 1):
        #embedding_dim is the same as input_dim as embeddings are used as input to the LSTM
        
        super(LSTM_special, self).__init__()
        self.input_dim= embedding_dim
        self.hidden_dim= hidden_dim
        self.batch_size= batch_size
        self.add_dim= add_dim
        self.out_dim= out_dim
        self.num_layers= num_layers
        
        # Embedding layer
        self.word_embeddings= nn.Embedding(vocab_size, self.input_dim)
        
        # Define the LSTM layer
        self.lstm= nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        
        # Define the linear layer that maps from (hidden_dim + add_dim) to out_dim
        self.linear= nn.Linear(self.hidden_dim+self.add_dim, self.out_dim)
        
        # Define the non-linearity that converts to probability
        self.softmax= nn.Softmax()
        
    def init_hidden(self):
        """
        Initialize the hidden state (h0, c0)
        
        Before we've done anything, we dont have any hidden state.
        Refer to the Pytorch documentation to see exactly
        why they have this dimensionality.
        The axes semantics are (num_layers, minibatch_size, hidden_dim)
        """
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
               torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
        
    def forward(self, sentence, add_features):
        """
        forward pass through LSTM layer
        first pass through the embedding layer
        hidden to output space mapping by Linear layer
        lstm_out shape: [seq_len/input_len, batch_size, hidden_dim]
        self.hidden shape= (a, b) where a & b both have shape: [num_layers, batch_size, hidden_dim]
        """
        
        # Embedding layer
        embeds= self.word_embeddings(sentence)
        
        
        lstm_out, self.hidden= self.lstm(embeds.view(len(sentence), self.batch_size, -1), self.hidden)
        
        """
        Take the output from the last layer of the LSTM and 
        concatenate the additional features to them.
        Map them to output space.
        Apply non linearity like softmax
        """
        # get the output from the last timestep
        lstm_out= lstm_out[-1].view(self.batch_size, -1)
        
        # concatenate additional features to lstm output
        new_features= torch.cat((lstm_out, add_features.view(self.batch_size, -1)), 1)
        
        # map to output space
        y_pred= self.linear(new_features)
        
        # apply non linearity
        output= self.softmax(y_pred)
        
        
        return output
        
        
        
        


In [37]:
"""
Initialize the model
Train the LSTM
"""

#################
# initialize the model
#################

model= LSTM_special(embedding_dim= 50, hidden_dim= 30, batch_size= 1, vocab_size= len(word_to_ix), add_dim= 5,) 

#################
# define the loss function
#################

loss_fn= torch.nn.MSELoss()
optimiser= torch.optim.Adam(model.parameters(), lr= 0.01)
num_epochs= 4

###################
#Train the model
###################

hist= np.zeros(num_epochs)

regex= re.compile(r"\b(\w*['\w]*[\w]*)[^\w]*")


for i in range(0, num_epochs):
    
    for ii, row in train_df.iterrows():
        
        #clear the stored gradients
        model.zero_grad()
        
        #clear the history of LSTM. Reset the hidden state
        model.hidden= model.init_hidden()
        
        # get the inputs ready for lstm
        sent= train_df.loc[ii, 'Question Text'].lower()
        sentence= prepare_seq(regex.findall(sent), word_to_ix)
        
        #additional features (time invariant)
        additional_features= [train_df.loc[ii, f] for f in features]
        additional_features= torch.tensor(additional_features)
        #additional_features= additional_features.type(torch.FloatTensor)
        
        #print((additional_features.values))
        #print((additional_features.values.view(1, -1)))
        
        # forward pass
        pred= model(sentence, additional_features)
        
        ans= torch.tensor(row['paren_match'])
        ans= ans.type(torch.FloatTensor).view(1,1)
        
        loss= loss_fn(pred, ans)
        
        hist[i]= loss.item()
        
        loss.backward()
    
        optimiser.step()
        print('Epoch: {}, row: {}'.format(i, ii))
        
    print('Epoch: ', i, 'MSE: ', loss.item())
        
        
        
        
        
        

/Users/akshatpant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0, row: 0
Epoch: 0, row: 1
Epoch: 0, row: 2
Epoch: 0, row: 3
Epoch: 0, row: 4
Epoch: 0, row: 5
Epoch: 0, row: 6
Epoch: 0, row: 7
Epoch: 0, row: 8
Epoch: 0, row: 9
Epoch: 0, row: 10
Epoch: 0, row: 11
Epoch: 0, row: 12
Epoch: 0, row: 13
Epoch: 0, row: 14
Epoch: 0, row: 15
Epoch: 0, row: 16
Epoch: 0, row: 17
Epoch: 0, row: 18
Epoch: 0, row: 19
Epoch: 0, row: 20
Epoch: 0, row: 21
Epoch: 0, row: 22
Epoch: 0, row: 23
Epoch: 0, row: 24
Epoch: 0, row: 25
Epoch: 0, row: 26
Epoch: 0, row: 27
Epoch: 0, row: 28
Epoch: 0, row: 29
Epoch: 0, row: 30
Epoch: 0, row: 31
Epoch: 0, row: 32
Epoch: 0, row: 33
Epoch: 0, row: 34
Epoch: 0, row: 35
Epoch: 0, row: 36
Epoch: 0, row: 37
Epoch: 0, row: 38
Epoch: 0, row: 39
Epoch: 0, row: 40
Epoch: 0, row: 41
Epoch: 0, row: 42
Epoch: 0, row: 43
Epoch: 0, row: 44
Epoch: 0, row: 45
Epoch: 0, row: 46
Epoch: 0, row: 47
Epoch: 0, row: 48
Epoch: 0, row: 49
Epoch: 0, row: 50
Epoch: 0, row: 51
Epoch: 0, row: 52
Epoch: 0, row: 53
Epoch: 0, row: 54
Epoch: 0, row: 55
Ep

Epoch: 0, row: 437
Epoch: 0, row: 438
Epoch: 0, row: 439
Epoch: 0, row: 440
Epoch: 0, row: 441
Epoch: 0, row: 442
Epoch: 0, row: 443
Epoch: 0, row: 444
Epoch: 0, row: 445
Epoch: 0, row: 446
Epoch: 0, row: 447
Epoch: 0, row: 448
Epoch: 0, row: 449
Epoch: 0, row: 450
Epoch: 0, row: 451
Epoch: 0, row: 452
Epoch: 0, row: 453
Epoch: 0, row: 454
Epoch: 0, row: 455
Epoch: 0, row: 456
Epoch: 0, row: 457
Epoch: 0, row: 458
Epoch: 0, row: 459
Epoch: 0, row: 460
Epoch: 0, row: 461
Epoch: 0, row: 462
Epoch: 0, row: 463
Epoch: 0, row: 464
Epoch: 0, row: 465
Epoch: 0, row: 466
Epoch: 0, row: 467
Epoch: 0, row: 468
Epoch: 0, row: 469
Epoch: 0, row: 470
Epoch: 0, row: 471
Epoch: 0, row: 472
Epoch: 0, row: 473
Epoch: 0, row: 474
Epoch: 0, row: 475
Epoch: 0, row: 476
Epoch: 0, row: 477
Epoch: 0, row: 478
Epoch: 0, row: 479
Epoch: 0, row: 480
Epoch: 0, row: 481
Epoch: 0, row: 482
Epoch: 0, row: 483
Epoch: 0, row: 484
Epoch: 0, row: 485
Epoch: 0, row: 486
Epoch: 0, row: 487
Epoch: 0, row: 488
Epoch: 0, ro

Epoch: 0, row: 874
Epoch: 0, row: 875
Epoch: 0, row: 876
Epoch: 0, row: 877
Epoch: 0, row: 878
Epoch: 0, row: 879
Epoch: 0, row: 880
Epoch: 0, row: 881
Epoch: 0, row: 882
Epoch: 0, row: 883
Epoch: 0, row: 884
Epoch: 0, row: 885
Epoch: 0, row: 886
Epoch: 0, row: 887
Epoch: 0, row: 888
Epoch: 0, row: 889
Epoch: 0, row: 890
Epoch: 0, row: 891
Epoch: 0, row: 892
Epoch: 0, row: 893
Epoch: 0, row: 894
Epoch: 0, row: 895
Epoch: 0, row: 896
Epoch: 0, row: 897
Epoch: 0, row: 898
Epoch: 0, row: 899
Epoch: 0, row: 900
Epoch: 0, row: 901
Epoch: 0, row: 902
Epoch: 0, row: 903
Epoch: 0, row: 904
Epoch: 0, row: 905
Epoch: 0, row: 906
Epoch: 0, row: 907
Epoch: 0, row: 908
Epoch: 0, row: 909
Epoch: 0, row: 910
Epoch: 0, row: 911
Epoch: 0, row: 912
Epoch: 0, row: 913
Epoch: 0, row: 914
Epoch: 0, row: 915
Epoch: 0, row: 916
Epoch: 0, row: 917
Epoch: 0, row: 918
Epoch: 0, row: 919
Epoch: 0, row: 920
Epoch: 0, row: 921
Epoch: 0, row: 922
Epoch: 0, row: 923
Epoch: 0, row: 924
Epoch: 0, row: 925
Epoch: 0, ro

Epoch: 0, row: 1298
Epoch: 0, row: 1299
Epoch: 0, row: 1300
Epoch: 0, row: 1301
Epoch: 0, row: 1302
Epoch: 0, row: 1303
Epoch: 0, row: 1304
Epoch: 0, row: 1305
Epoch: 0, row: 1306
Epoch: 0, row: 1307
Epoch: 0, row: 1308
Epoch: 0, row: 1309
Epoch: 0, row: 1310
Epoch: 0, row: 1311
Epoch: 0, row: 1312
Epoch: 0, row: 1313
Epoch: 0, row: 1314
Epoch: 0, row: 1315
Epoch: 0, row: 1316
Epoch: 0, row: 1317
Epoch: 0, row: 1318
Epoch: 0, row: 1319
Epoch: 0, row: 1320
Epoch: 0, row: 1321
Epoch: 0, row: 1322
Epoch: 0, row: 1323
Epoch: 0, row: 1324
Epoch: 0, row: 1325
Epoch: 0, row: 1326
Epoch: 0, row: 1327
Epoch: 0, row: 1328
Epoch: 0, row: 1329
Epoch: 0, row: 1330
Epoch: 0, row: 1331
Epoch: 0, row: 1332
Epoch: 0, row: 1333
Epoch: 0, row: 1334
Epoch: 0, row: 1335
Epoch: 0, row: 1336
Epoch: 0, row: 1337
Epoch: 0, row: 1338
Epoch: 0, row: 1339
Epoch: 0, row: 1340
Epoch: 0, row: 1341
Epoch: 0, row: 1342
Epoch: 0, row: 1343
Epoch: 0, row: 1344
Epoch: 0, row: 1345
Epoch: 0, row: 1346
Epoch: 0, row: 1347


...

Epoch: 3, row: 6444
Epoch: 3, row: 6445
Epoch: 3, row: 6446
Epoch: 3, row: 6447
Epoch: 3, row: 6448
Epoch: 3, row: 6449
Epoch: 3, row: 6450
Epoch: 3, row: 6451
Epoch: 3, row: 6452
Epoch: 3, row: 6453
Epoch: 3, row: 6454
Epoch: 3, row: 6455
Epoch: 3, row: 6456
Epoch: 3, row: 6457
Epoch: 3, row: 6458
Epoch: 3, row: 6459
Epoch: 3, row: 6460
Epoch: 3, row: 6461
Epoch: 3, row: 6462
Epoch: 3, row: 6463
Epoch: 3, row: 6464
Epoch: 3, row: 6465
Epoch: 3, row: 6466
Epoch: 3, row: 6467
Epoch: 3, row: 6468
Epoch: 3, row: 6469
Epoch: 3, row: 6470
Epoch: 3, row: 6471
Epoch: 3, row: 6472
Epoch: 3, row: 6473
Epoch: 3, row: 6474
Epoch: 3, row: 6475
Epoch: 3, row: 6476
Epoch: 3, row: 6477
Epoch: 3, row: 6478
Epoch: 3, row: 6479
Epoch: 3, row: 6480
Epoch: 3, row: 6481
Epoch: 3, row: 6482
Epoch: 3, row: 6483
Epoch: 3, row: 6484
Epoch: 3, row: 6485
Epoch: 3, row: 6486
Epoch: 3, row: 6487
Epoch: 3, row: 6488
Epoch: 3, row: 6489
Epoch: 3, row: 6490
Epoch: 3, row: 6491
Epoch: 3, row: 6492
Epoch: 3, row: 6493


Epoch: 3, row: 6855
Epoch: 3, row: 6856
Epoch: 3, row: 6857
Epoch: 3, row: 6858
Epoch: 3, row: 6859
Epoch: 3, row: 6860
Epoch: 3, row: 6861
Epoch: 3, row: 6862
Epoch: 3, row: 6863
Epoch: 3, row: 6864
Epoch: 3, row: 6865
Epoch: 3, row: 6866
Epoch: 3, row: 6867
Epoch: 3, row: 6868
Epoch: 3, row: 6869
Epoch: 3, row: 6870
Epoch: 3, row: 6871
Epoch: 3, row: 6872
Epoch: 3, row: 6873
Epoch: 3, row: 6874
Epoch: 3, row: 6875
Epoch: 3, row: 6876
Epoch: 3, row: 6877
Epoch: 3, row: 6878
Epoch: 3, row: 6879
Epoch: 3, row: 6880
Epoch: 3, row: 6881
Epoch: 3, row: 6882
Epoch: 3, row: 6883
Epoch: 3, row: 6884
Epoch: 3, row: 6885
Epoch: 3, row: 6886
Epoch: 3, row: 6887
Epoch: 3, row: 6888
Epoch: 3, row: 6889
Epoch: 3, row: 6890
Epoch: 3, row: 6891
Epoch: 3, row: 6892
Epoch: 3, row: 6893
Epoch: 3, row: 6894
Epoch: 3, row: 6895
Epoch: 3, row: 6896
Epoch: 3, row: 6897
Epoch: 3, row: 6898
Epoch: 3, row: 6899
Epoch: 3, row: 6900
Epoch: 3, row: 6901
Epoch: 3, row: 6902
Epoch: 3, row: 6903
Epoch: 3, row: 6904


Epoch: 3, row: 7268
Epoch: 3, row: 7269
Epoch: 3, row: 7270
Epoch: 3, row: 7271
Epoch: 3, row: 7272
Epoch: 3, row: 7273
Epoch: 3, row: 7274
Epoch: 3, row: 7275
Epoch: 3, row: 7276
Epoch: 3, row: 7277
Epoch: 3, row: 7278
Epoch: 3, row: 7279
Epoch: 3, row: 7280
Epoch: 3, row: 7281
Epoch: 3, row: 7282
Epoch: 3, row: 7283
Epoch: 3, row: 7284
Epoch: 3, row: 7285
Epoch: 3, row: 7286
Epoch: 3, row: 7287
Epoch: 3, row: 7288
Epoch: 3, row: 7289
Epoch: 3, row: 7290
Epoch: 3, row: 7291
Epoch: 3, row: 7292
Epoch: 3, row: 7293
Epoch: 3, row: 7294
Epoch: 3, row: 7295
Epoch: 3, row: 7296
Epoch: 3, row: 7297
Epoch: 3, row: 7298
Epoch: 3, row: 7299
Epoch: 3, row: 7300
Epoch: 3, row: 7301
Epoch: 3, row: 7302
Epoch: 3, row: 7303
Epoch: 3, row: 7304
Epoch: 3, row: 7305
Epoch: 3, row: 7306
Epoch: 3, row: 7307
Epoch: 3, row: 7308
Epoch: 3, row: 7309
Epoch: 3, row: 7310
Epoch: 3, row: 7311
Epoch: 3, row: 7312
Epoch: 3, row: 7313
Epoch: 3, row: 7314
Epoch: 3, row: 7315
Epoch: 3, row: 7316
Epoch: 3, row: 7317


Epoch: 3, row: 7681
Epoch: 3, row: 7682
Epoch: 3, row: 7683
Epoch: 3, row: 7684
Epoch: 3, row: 7685
Epoch: 3, row: 7686
Epoch: 3, row: 7687
Epoch: 3, row: 7688
Epoch: 3, row: 7689
Epoch: 3, row: 7690
Epoch: 3, row: 7691
Epoch: 3, row: 7692
Epoch: 3, row: 7693
Epoch: 3, row: 7694
Epoch: 3, row: 7695
Epoch: 3, row: 7696
Epoch: 3, row: 7697
Epoch: 3, row: 7698
Epoch: 3, row: 7699
Epoch: 3, row: 7700
Epoch: 3, row: 7701
Epoch: 3, row: 7702
Epoch: 3, row: 7703
Epoch: 3, row: 7704
Epoch: 3, row: 7705
Epoch: 3, row: 7706
Epoch: 3, row: 7707
Epoch: 3, row: 7708
Epoch: 3, row: 7709
Epoch: 3, row: 7710
Epoch: 3, row: 7711
Epoch: 3, row: 7712
Epoch: 3, row: 7713
Epoch: 3, row: 7714
Epoch: 3, row: 7715
Epoch: 3, row: 7716
Epoch: 3, row: 7717
Epoch: 3, row: 7718
Epoch: 3, row: 7719
Epoch: 3, row: 7720
Epoch: 3, row: 7721
Epoch: 3, row: 7722
Epoch: 3, row: 7723
Epoch: 3, row: 7724
Epoch: 3, row: 7725
Epoch: 3, row: 7726
Epoch: 3, row: 7727
Epoch: 3, row: 7728
Epoch: 3, row: 7729
Epoch: 3, row: 7730


Epoch: 3, row: 8092
Epoch: 3, row: 8093
Epoch: 3, row: 8094
Epoch: 3, row: 8095
Epoch: 3, row: 8096
Epoch: 3, row: 8097
Epoch: 3, row: 8098
Epoch: 3, row: 8099
Epoch: 3, row: 8100
Epoch: 3, row: 8101
Epoch: 3, row: 8102
Epoch: 3, row: 8103
Epoch: 3, row: 8104
Epoch: 3, row: 8105
Epoch: 3, row: 8106
Epoch: 3, row: 8107
Epoch: 3, row: 8108
Epoch: 3, row: 8109
Epoch: 3, row: 8110
Epoch: 3, row: 8111
Epoch: 3, row: 8112
Epoch: 3, row: 8113
Epoch: 3, row: 8114
Epoch: 3, row: 8115
Epoch: 3, row: 8116
Epoch: 3, row: 8117
Epoch: 3, row: 8118
Epoch: 3, row: 8119
Epoch: 3, row: 8120
Epoch: 3, row: 8121
Epoch: 3, row: 8122
Epoch: 3, row: 8123
Epoch: 3, row: 8124
Epoch: 3, row: 8125
Epoch: 3, row: 8126
Epoch: 3, row: 8127
Epoch: 3, row: 8128
Epoch: 3, row: 8129
Epoch: 3, row: 8130
Epoch: 3, row: 8131
Epoch: 3, row: 8132
Epoch: 3, row: 8133
Epoch: 3, row: 8134
Epoch: 3, row: 8135
Epoch: 3, row: 8136
Epoch: 3, row: 8137
Epoch: 3, row: 8138
Epoch: 3, row: 8139
Epoch: 3, row: 8140
Epoch: 3, row: 8141


In [38]:
for ii, row in train_df.iterrows():
        
    #clear the stored gradients
    model.zero_grad()
        
    #clear the history of LSTM. Reset the hidden state
    model.hidden= model.init_hidden()
        
    # get the inputs ready for lstm
    sent= train_df.loc[ii, 'Question Text'].lower()
    sentence= prepare_seq(regex.findall(sent), word_to_ix)
        
    #additional features (time invariant)
    additional_features= [train_df.loc[ii, f] for f in features]
    additional_features= torch.tensor(additional_features)
    #additional_features= additional_features.type(torch.FloatTensor)
        
    #print((additional_features.values))
    #print((additional_features.values.view(1, -1)))
        
    # forward pass
    pred= model(sentence, additional_features)
    print(pred)

/Users/akshatpant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBac

tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBac

tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBac

tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBac

...

tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBac

tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBac

tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBac

tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBac

tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBackward>)
tensor([[1.]], grad_fn=<SoftmaxBac

In [175]:


a= [1,2,3,4]
a= torch.tensor(a)
a= a.type(torch.FloatTensor)
a= (a.view(1, -1))
print(a)

out, h= model.lstm(X_train[0].view(seq_len, model.batch_size, -1), model.hidden)
#print(out[-1].view(-1))

print(torch.cat((out[-1].view(1, -1), a), 1))



tensor([[1., 2., 3., 4.]])


KeyError: 0